In [13]:
from pandas import DataFrame, read_csv

data = read_csv('/Users/catcamacho/Box/LCBD/ChEC/jumble/Jumble_combined_AOI_cleaned_20200124.csv',index_col=None)
data.columns

Index(['Trial', 'Stimulus', 'Emotion', 'Type', 'Export Start Trial Time [ms]',
       'Export End Trial Time [ms]', 'Participant', 'Color', 'Eye L/R',
       'AOI Name', 'AOI Scope', 'AOI Order', 'AOI Size [px]',
       'AOI Coverage [%]', 'Appearance Count', 'Visible Time [ms]',
       'Visible Time [%]', 'Entry Time [ms]', 'Sequence',
       'Net Dwell Time [ms]', 'Dwell Time [ms]',
       'Normalized Dwell [ms/Coverage]', 'Glance Duration [ms]',
       'Diversion Duration [ms]', 'First Fixation Duration [ms]',
       'Glances Count', 'Revisits', 'Fixation Count', 'Net Dwell Time [%]',
       'Dwell Time [%]', 'Fixation Time [ms]', 'Fixation Time [%]',
       'Time to First Mouse Click [ms]', 'Average Fixation [ms]'],
      dtype='object')

In [ ]:
import numpy as np

subs = np.unique(data['Participant'])
emotion = np.unique(data['Emotion'])
trial_type = np.unique(data['Type'])
columns = []

for emo in emotion:
    emo_t = data[data['Emotion']==emo]
    for trial in trial_type:
        trial_t = emo_t[emo_t['Type']==trial]
        aois = np.unique(trial_t['AOI Name'])
        for a in aois:
            columns.append(emo+'_'+trial+'_'+a+'_rawdwell')
            columns.append(emo+'_'+trial+'_'+a+'_normdwell')
            columns.append(emo+'_'+trial+'_'+a+'_pcentdwell')
            columns.append(emo+'_'+trial+'_'+a+'_entrytime')

In [15]:
import numpy as np

subs = np.unique(data['Participant'])
emotion = np.unique(data['Emotion'])
trial_type = np.unique(data['Type'])

summary_data = DataFrame(columns=columns,index=subs)

for sub in subs:
    temp = data[data['Participant']==sub]
    for emo in emotion:
        emo_t = temp[temp['Emotion']==emo]
        for trial in trial_type:
            trial_t = emo_t[emo_t['Type']==trial]
            aois = np.unique(trial_t['AOI Name'])
            for a in aois:
                a_t = trial_t[trial_t['AOI Name']==a]
                if a=='eye':
                    stims = np.unique(a_t['Stimulus'])
                    comb_eye = DataFrame(columns=['dwelltime','normdwell','normpcent','entry'], index=stims)
                    for stim in stims:
                        a_t_s = a_t[a_t['Stimulus']==stim]
                        aoi_num = np.unique(a_t_s['AOI Order'])
                        a_t_s = a_t_s.groupby(['AOI Order']).mean()
                        comb_eye.loc[stim,'dwelltime'] = a_t_s.loc[aoi_num[0],'Dwell Time [ms]'] + a_t_s.loc[aoi_num[1],'Dwell Time [ms]']
                        comb_eye.loc[stim,'normdwell'] = a_t_s.loc[aoi_num[0],'Normalized Dwell [ms/Coverage]'] + a_t_s.loc[aoi_num[1],'Normalized Dwell [ms/Coverage]']
                        comb_eye.loc[stim,'normpcent'] = a_t_s.loc[aoi_num[0],'Dwell Time [%]'] + a_t_s.loc[aoi_num[1],'Dwell Time [%]']
                        comb_eye.loc[stim,'entry'] = a_t_s['Entry Time [ms]'].min()
                        
                    summary_data.loc[sub,emo+'_'+trial+'_'+a+'_rawdwell'] = comb_eye['dwelltime'].mean()
                    summary_data.loc[sub,emo+'_'+trial+'_'+a+'_normdwell'] = comb_eye['normdwell'].mean()
                    summary_data.loc[sub,emo+'_'+trial+'_'+a+'_pcentdwell'] = comb_eye['normpcent'].mean()
                    summary_data.loc[sub,emo+'_'+trial+'_'+a+'_entrytime'] = comb_eye['entry'].mean()
                else:
                    summary_data.loc[sub,emo+'_'+trial+'_'+a+'_rawdwell'] = a_t['Dwell Time [ms]'].mean()
                    summary_data.loc[sub,emo+'_'+trial+'_'+a+'_normdwell'] = a_t['Normalized Dwell [ms/Coverage]'].mean()
                    summary_data.loc[sub,emo+'_'+trial+'_'+a+'_pcentdwell'] = a_t['Dwell Time [%]'].mean()
                    summary_data.loc[sub,emo+'_'+trial+'_'+a+'_entrytime'] = a_t['Entry Time [ms]'].mean()
                
                
summary_data.to_csv('summarized_eyetracking_20200124.csv')       